# To do

# Configurando ambiente

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')
import os
import pandas as pd
import numpy as np
import glob

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
pd.set_option('display.max_rows', None)

In [3]:
path = '/content/gdrive/My Drive/br_ms_sinan/'

# Criando pastas particionadas

In [4]:
#lê todos os arquivos (separados por ano)
file = []
for filepath in glob.iglob(path + r'input/*.csv'):
    file.append(filepath)

#cria dicionário de ano atrelado ao arquivo
ano=[]
for i in range(len(file)) : ano.append(int(file[i][-8:-4]))

dict_ano = dict(zip(ano, file))
dict_ano

#UF
ufs = ['AC', 'AL', 'AM', 'AP', 'BA', 'CE', 'DF', 'ES', 'GO', 'MA', 'MG', 'MS', 'MT', 
       'PA', 'PB', 'PE', 'PI', 'PR', 'RJ', 'RN', 'RO', 'RR', 'RS', 'SC', 'SE', 'SP', 'TO'] 

for i in [*range(2009, 2023)]:
  for uf in ufs:    
    partition = path + 'output/ano={}/sigla_uf={}'.format(i, uf)
    if not os.path.exists(partition): # só vai criar se não tiver a pasta
      os.makedirs(partition) # comando que cria as pastas

#repara que o caminho da partição, é a sua pasta no drive com + um complemento do output, tu pode usar esse formato pra parte do salvando as bases

# Abrindo, limpando e salvando bases ano a ano

In [5]:
#tabela complementar com informações de id_municipio com 7 dígitos já que o MS oferece somente com 6 dígitos
id_mun_7 = pd.read_csv(path + 'input/dados_complementares/id_mun.csv', dtype='string')
id_mun_7_2 = pd.read_csv(path + 'input/dados_complementares/id_mun.csv', dtype='string')
id_mun_7_3 = pd.read_csv(path + 'input/dados_complementares/id_mun.csv', dtype='string')

In [6]:
id_mun_7['id_municipio_6_notificacao'] = id_mun_7['id_municipio'].str.slice(stop=6)
id_mun_7 = id_mun_7.rename(columns = {"id_municipio": "id_municipio_notificacao"})

id_mun_7_2['id_municipio_6_residencia'] = id_mun_7_2['id_municipio'].str.slice(stop=6)
id_mun_7_2 = id_mun_7_2.rename(columns = {"id_municipio": "id_municipio_residencia"})

id_mun_7_3['id_municipio_6_internacao'] = id_mun_7_3['id_municipio'].str.slice(stop=6)
id_mun_7_3 = id_mun_7_3.rename(columns = {"id_municipio": "id_municipio_internacao"})

In [7]:
#tabela complementar com infos sobre id e siglas uf da BD
sigla_uf   = pd.read_csv(path + 'input/dados_complementares/id_municipio_siglauf.csv', dtype='string')
sigla_uf_2 = pd.read_csv(path + 'input/dados_complementares/id_municipio_siglauf.csv', dtype='string')
sigla_uf_3 = pd.read_csv(path + 'input/dados_complementares/id_municipio_siglauf.csv', dtype='string')

In [8]:
#sheet_url = 'https://docs.google.com/spreadsheets/d/1WNL4NbYrxQHnzx5drtkR98RjqR6iqvrXpCPO3Kknxxs/edit#gid=1805343824'
#url_1 = sheet_url.replace('/edit#gid=', '/export?format=csv&gid=')

In [9]:
# ordem 
df_arquitetura = pd.read_csv(path + '/input/dados_complementares/[arquitetura] br_ms_sinan.influenza_srag - Sheet1.csv', dtype='string')
#df_arquitetura = pd.read_csv(url_1, dtype='string') (mudar aqui caso a gente altere algo na arquitetura)

In [10]:
ordem = list(df_arquitetura.name)

In [11]:
add_df = pd.DataFrame(columns=ordem)

In [12]:
#renomeia as variáveis no padrão BD usando a arquitetura para cada ano

for year in range(2009, 2023):
    column = 'original_name_{}'.format(year)
    rename_name = 'rename_{}'.format(year)
    df_filtered = df_arquitetura.loc[pd.notna(df_arquitetura[column])]
    globals()[rename_name] = df_filtered.set_index(column)['name'].to_dict()

In [13]:
# cria dataframes pra compatibilização da base completa/ordenação
dict_list = [globals()[f'rename_{year}'] for year in range(2009, 2023)]

In [14]:
df_ordem = pd.DataFrame(columns = ordem)

In [15]:
add_dfs = [df_ordem.drop([value for key, value in dic.items() if value in df_ordem.columns], axis=1) for dic in dict_list]

In [16]:
for i, df in enumerate(add_dfs):
    year = 2009 + i
    df_name = "add_{}".format(year)
    globals()[df_name] = df.loc[:,~df.columns.duplicated()].copy()

# Código Tratamento

In [17]:
for i in ano:

        exec('{} = pd.DataFrame()'.format('sinan_' + str(i)))
        ch=0
        for chunk in pd.read_csv(dict_ano[i],sep=',', chunksize = 100000, nrows = 10000, dtype='string'):
            ch = ch + 1
            print("fazendo chunk {} de {} ".format(ch,i))
            exec('chunk.rename(columns = {}, inplace=True)'.format('rename_' + str(i)))         
            exec('chunk = pd.concat([chunk, {}])'.format('add_' + str(i)))
            cols = chunk.filter(like='data').columns
            for col in cols:
                chunk[col] = pd.to_datetime(chunk[col], format='%d/%m/%Y', errors='coerce')
            chunk['ano'] = chunk['data_notificacao'].dt.year
            for col in cols:
                chunk[col] = chunk[col].dt.date

            chunk = chunk.merge(id_mun_7, on='id_municipio_6_notificacao', how='left')
            chunk = chunk.rename(columns={'id_municipio_notificacao_y': 'id_municipio_notificacao'})
            chunk = chunk.merge(id_mun_7_2, on='id_municipio_6_residencia', how='left')
            chunk = chunk.rename(columns={'id_municipio_residencia_y': 'id_municipio_residencia'})
            chunk['id_municipio_6_internacao'] = chunk['id_municipio_6_internacao'].str.slice(stop=6)
            chunk = chunk.merge(id_mun_7_3, on='id_municipio_6_internacao', how='left')
            chunk = chunk.rename(columns={'id_municipio_internacao_y': 'id_municipio_internacao'})

            if not chunk['sigla_uf_notificacao'].str.isalpha().all():
              chunk = pd.merge(chunk, sigla_uf[['sigla_uf', 'id_municipio']], how='left', left_on='id_municipio_notificacao', right_on='id_municipio')
              chunk['sigla_uf_notificacao'] = chunk['sigla_uf']
              chunk = chunk.drop(columns = "sigla_uf")
              chunk = pd.merge(chunk, sigla_uf[['sigla_uf', 'id_municipio']], how='left', left_on='id_municipio_residencia', right_on='id_municipio')
              chunk['sigla_uf_residencia'] = chunk['sigla_uf']
              chunk = chunk.drop(columns = "sigla_uf")
              chunk = pd.merge(chunk, sigla_uf[['sigla_uf', 'id_municipio']], how='left', left_on='id_municipio_internacao', right_on='id_municipio')
              chunk['sigla_uf_internacao'] = chunk['sigla_uf']
            
            chunk['semana_notificacao'] = chunk['semana_notificacao'].str[-2:]
            chunk['semana_sintomas'] = chunk['semana_sintomas'].str[-2:]

            #float_cols = chunk.select_dtypes(include=['float']).columns
            #chunk[float_cols] = chunk[float_cols].astype(int)

            if i <= 2011:
              chunk['sistema'] = 'sinan_net'
            elif i <= 2018:
              chunk['sistema'] = "sinan_influenza_web"
            else:
              chunk['sistema'] = "sivep_gripe"

            chunk = chunk[ordem]
            exec('{} = {}.append(chunk)'.format('sinan_' + str(i), 'sinan_' + str(i)))
        exec('{}.to_csv(path + "tmp/{}.csv",index=False, encoding="utf-8", na_rep="", date_format="%Y-%m-%d")'.format('sinan_' + str(i), 'sinan_' + str(i)))
        exec('del {}'.format('sinan_' + str(i)))

fazendo chunk 1 de 2009 


<string>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.


fazendo chunk 1 de 2011 


<string>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.


fazendo chunk 1 de 2010 


...

<string>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.


fazendo chunk 4 de 2022 


<string>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.


fazendo chunk 5 de 2022 


<string>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.


# Salvando particionado


In [18]:
temp = []
for filepath in glob.iglob(path + r'tmp/*.csv'):
    temp.append(filepath)

anot=[]
for i in range(len(temp)) : anot.append(int(temp[i][-8:-4]))

dict_anot = dict(zip(anot, temp))
dict_anot

{2009: '/content/gdrive/My Drive/br_ms_sinan/tmp/sinan_2009.csv',
 2011: '/content/gdrive/My Drive/br_ms_sinan/tmp/sinan_2011.csv',
 2010: '/content/gdrive/My Drive/br_ms_sinan/tmp/sinan_2010.csv',
 2012: '/content/gdrive/My Drive/br_ms_sinan/tmp/sinan_2012.csv',
 2013: '/content/gdrive/My Drive/br_ms_sinan/tmp/sinan_2013.csv',
 2014: '/content/gdrive/My Drive/br_ms_sinan/tmp/sinan_2014.csv',
 2015: '/content/gdrive/My Drive/br_ms_sinan/tmp/sinan_2015.csv',
 2016: '/content/gdrive/My Drive/br_ms_sinan/tmp/sinan_2016.csv',
 2017: '/content/gdrive/My Drive/br_ms_sinan/tmp/sinan_2017.csv',
 2018: '/content/gdrive/My Drive/br_ms_sinan/tmp/sinan_2018.csv',
 2019: '/content/gdrive/My Drive/br_ms_sinan/tmp/sinan_2019.csv',
 2020: '/content/gdrive/My Drive/br_ms_sinan/tmp/sinan_2020.csv',
 2021: '/content/gdrive/My Drive/br_ms_sinan/tmp/sinan_2021.csv',
 2022: '/content/gdrive/My Drive/br_ms_sinan/tmp/sinan_2022.csv'}

In [19]:
for i in anot:
  for uf in ufs: 
        exec('{} = pd.read_csv(path + "tmp/sinan_{}.csv", sep=",", dtype="string")'.format('sinan_' + str(i), str(i)))
        exec("{}.sort_values(['data_notificacao', 'id_municipio_notificacao'], inplace=True)".format('sinan_' + str(i)))
        exec("{}.drop_duplicates(keep='first', inplace=True)".format('sinan_' + str(i))) 
        exec("{} = {}[{}['sigla_uf_notificacao']== uf]".format('sinan_' + str(i), 'sinan_' + str(i), 'sinan_' + str(i)))
        exec("{}.drop(['sigla_uf_notificacao','ano'], axis=1, inplace=True)".format('sinan_' + str(i)))
        print("Particionando {} de {}".format(uf, str(i)))
        exec('{}.to_csv(path + "output/ano={}/sigla_uf={}/microdados.csv",index=False, encoding="utf-8", na_rep="", date_format="%Y-%m-%d")'.format('sinan_' + str(i), str(i), uf))
        exec('del {}'.format('sinan_' + str(i)))

<string>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Particionando AC de 2009


<string>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Particionando AL de 2009


<string>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


...

Particionando SE de 2022


<string>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Particionando SP de 2022


<string>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Particionando TO de 2022
